# How is Illinois housing inventory trending over time?
#### Methodology
1. Extract, combine, and clean data
2. Calculate average percentages of new listings 
4. Create scatter plot all counties over time 

### Import Libraries and Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import fredapi as fa
import seaborn as sns

api_key = '4326c34340957958ddf39343ad409053'

new_listings_data = pd.read_csv(r'C:/Users/kevin/Documents/analysis_docs/Illinois_Housing_Affordability/csv_files/Counties.csv')
fred = fa.Fred(api_key=api_key)

# Create an empty list to store DataFrames
listings_dataframes = []

# Loop through county data and fetch data from FRED
for index, row in new_listings_data.iterrows():
    county_name = row['county_name']
    county_id = row['new_listings']

    try:
        list_data = fred.get_series(county_id)
        list_data = list_data.to_frame(name='new_listings')  # Ensure data is in a DataFrame       
        list_data['county_name'] = county_name
        list_data['month'] = list_data.index.strftime('%b')
        list_data['year'] = list_data.index.year
        listings_dataframes.append(list_data)
    except Exception as e:
        print(f"Error retrieving data for {county_name}: {e}")

# Combine DataFrames
combined_listings_df = pd.concat(listings_dataframes)
combined_listings_df = combined_listings_df.reset_index()
combined_listings_df.rename(columns={'index': 'Date'}, inplace=True)


print(combined_listings_df)

           Date  new_listings           county_name month  year
0    2017-07-01          2.70      Adams County, IL   Jul  2017
1    2017-08-01          0.00      Adams County, IL   Aug  2017
2    2017-09-01          5.26      Adams County, IL   Sep  2017
3    2017-10-01         10.00      Adams County, IL   Oct  2017
4    2017-11-01        -27.27      Adams County, IL   Nov  2017
...         ...           ...                   ...   ...   ...
2800 2024-03-01         31.31  Winnebago County, IL   Mar  2024
2801 2024-04-01         -1.54  Winnebago County, IL   Apr  2024
2802 2024-05-01         23.44  Winnebago County, IL   May  2024
2803 2024-06-01          6.33  Winnebago County, IL   Jun  2024
2804 2024-07-01         -5.95  Winnebago County, IL   Jul  2024

[2805 rows x 5 columns]


### Calculate Average Percentage of New Listings by County

In [3]:
# Group by 'county_name' and calculate the mean of 'new_listings'
average_new_listings_per_county = combined_listings_df.groupby(['county_name', 'year'])['new_listings'].mean()
anl_df = average_new_listings_per_county.to_frame(name='new_listings')

### Plot Average Percentage of New Listings by County over Time

In [ ]:
sns.set_style("whitegrid")
# Create the scatter plot
sns.scatterplot(x='year', y='new_listings', data=anl_df, alpha=0.7)
plt.xlabel('year')
plt.ylabel('New Listings (%)')
plt.title('Average Percentage of New Listings by County Over Time')
sns.despine() 
plt.grid(True)
# Format y axis with %
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, p: f"{y:.0f}%"))
plt.show()

![Avg Percentage of New Listings by County over Time](images\Avg-Percentage-New-Listings-Over-Time.png)